# TMA4220 Project 2
Group 10: Nikolai Rasmus Sætren, August Hardersen Skogøy

In [1]:
import ngsolve
from netgen.geom2d import unit_square
import netgen.gui
from netgen.meshing import *
from ngsolve.webgui import Draw
import numpy as np
from ngsolve import dx, grad, Redraw,x,y,cos,sin,exp,pi

%gui tk

$$
\begin{align*}
(\partial_t \phi, v) + (M \nabla \mu, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu, w) - \left( \gamma (\phi^3 - \phi), w \right) - \left( \epsilon \nabla \phi, \nabla w \right) &= 0 \quad \text{for all } w \in V.
\end{align*}

$$

In [2]:
# Defining constants

epsilon = 0.02
gamma = 50
M = 1

h_max = 0.02

In [3]:
# Setting up the mesh

def genMesh(h_max):

    ngmesh = Mesh(dim=2)

    N= int(1/h_max)

    """
    Code for the square mesh taken from the documentation page
    https://docu.ngsolve.org/latest/i-tutorials/unit-4.3-manualmesh/manualmeshing.html
    """


    pnums = []
    for i in range(N + 1):
        for j in range(N + 1):
            pnums.append(ngmesh.Add(MeshPoint(Pnt(i / N, j / N, 0))))

    idx_dom = ngmesh.AddRegion("mat", dim=2)
    for j in range(N):
        for i in range(N):
            ngmesh.Add(Element2D(idx_dom, [pnums[i + j * (N + 1)],
                                pnums[i + (j + 1) * (N + 1)],
                                pnums[i + 1 + (j + 1) * (N + 1)],
                                pnums[i + 1 + j * (N + 1)]]))
            
    # horizontal boundaries
    for i in range(N):
        ngmesh.Add(Element1D([pnums[N + i * (N + 1)],
                            pnums[N + (i + 1) * (N + 1)]], index=1))
        ngmesh.Add(Element1D([pnums[0 + i * (N + 1)], pnums[0 + (i + 1) * (N + 1)]], index=1))

    # vertical boundaries
    for i in range(N):
        ngmesh.Add(Element1D([pnums[i], pnums[i + 1]], index=2))
        ngmesh.Add(Element1D([pnums[i + N * (N + 1)], pnums[i + 1 + N * (N + 1)]], index=2))

    mesh = ngsolve.Mesh(ngmesh)
    return mesh

Fully explicit:
$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - (\epsilon \nabla \phi^{n+1}, \nabla w) &= \left( \gamma ((\phi^n)^3 - \phi^n), w \right) \quad \text{for all } w \in V.
\end{align*}

$$

which can be reformulated as

$$
\begin{align*}
\left( \frac{\phi^{n+1} }{\tau}, v \right) &=  \left( \frac{\phi^n}{\tau}, v \right) - (M \nabla \mu^{n+1}, \nabla v)  \quad \text{for all } v \in V, \\
(\mu^{n+1}, w)  - (\epsilon \nabla \phi^{n+1}, \nabla w) &= + \left( \gamma ((\phi^n)^3 - \phi^n), w \right) \quad \text{for all } w \in V.
\end{align*}
$$

In [4]:
# Fully Explicit Scheme

def fullyExplicit(h_max,tau, iter, fix_seed=False):
    mesh= genMesh(h_max)

    # Defining finite element space

    V_phi = ngsolve.H1(mesh,order=1)
    V_mu  = ngsolve.H1(mesh,order=1)

    V = ngsolve.FESpace([V_phi, V_mu])  # Mixed finite element space
    (u_phi, u_mu), (v_phi, v_mu) = V.TnT()  # Trial and test functions

    gfu_phi = ngsolve.GridFunction(V_phi)
    gfu_mu = ngsolve.GridFunction(V_mu)


    # Initialisation of phi_0

    if fix_seed:
        np.random.seed(0)
    phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
    rand_expr = ngsolve.GridFunction(V_phi)
    for i in range(mesh.nv):
        rand_expr.vec[i] = phi_0[i]

    gfu_phi.Set(rand_expr) 


    # Bilinear form

    def assemble_A(gfu_phi):
        A = ngsolve.BilinearForm(V)
        A += (1 / tau) * u_phi * v_phi * dx  
        A += M * grad(u_mu) * grad(v_phi) * dx  
        A += u_mu * v_mu * dx  
        A += -epsilon * grad(u_phi) * grad(v_mu) * dx   
        A.Assemble()
        return A

    # linear form

    def assemble_F(gfu_phi):
        F = ngsolve.LinearForm(V)
        F += (1 / tau) * gfu_phi * v_phi * dx  
        F += gamma * ((gfu_phi**3 - gfu_phi)* v_mu) * dx 
        F.Assemble()
        return F

    gfu = ngsolve.GridFunction(V)  # Mixed solution vector [phi, mu]


    Draw(gfu_phi)
    # Time-stepping loop
    for n in range(iter):
        # Update RHS
        F = assemble_F(gfu_phi)
        A = assemble_A(gfu_phi)
        
        # Solve the coupled system
        gfu.vec.data = A.mat.Inverse(V.FreeDofs()) * F.vec

        # Extract phi and mu components
        gfu_phi.vec.data = gfu.components[0].vec
        gfu_mu.vec.data = gfu.components[1].vec


    Draw(gfu_phi)





In [ ]:
h_max = 0.05
tau = 1e-5

fullyExplicit(h_max,tau,iter = 100, fix_seed =True)


    

Implicit-explicit (IMEX)

$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - \left( \gamma \left[(\phi^n)^2 - 1\right] \phi^{n+1}, w \right) - (\epsilon \nabla \phi^{n+1}, \nabla w) &= 0 \quad \text{for all } w \in V.
\end{align*}


$$


In [6]:
# IMEX

def IMEX(h_max, tau,iter,fix_seed = False):

    # Mesh
    mesh = genMesh(h_max)

    # Defining finite element space

    V_phi = ngsolve.H1(mesh,order=1)
    V_mu  = ngsolve.H1(mesh,order=1)

    V = ngsolve.FESpace([V_phi, V_mu])  # Mixed finite element space
    (u_phi, u_mu), (v_phi, v_mu) = V.TnT()  # Trial and test functions

    gfu_phi = ngsolve.GridFunction(V_phi)
    gfu_mu = ngsolve.GridFunction(V_mu)


    # Initialisation of phi_0

    if fix_seed:
        np.random.seed(0)
    phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
    rand_expr = ngsolve.GridFunction(V_phi)
    for i in range(mesh.nv):
        rand_expr.vec[i] = phi_0[i]

    gfu_phi.Set(rand_expr) 

    # Bilinear form

    def assemble_A(gfu_phi):
        A = ngsolve.BilinearForm(V)
        A += (1 / tau) * u_phi * v_phi * dx  
        A += M * grad(u_mu) * grad(v_phi) * dx  
        A += u_mu * v_mu * dx  
        A += -epsilon * grad(u_phi) * grad(v_mu) * dx  
        A += -gamma * ((gfu_phi**2 - 1) * u_phi * v_mu) * dx  
        A.Assemble()
        return A

    # linear form

    def assemble_F(gfu_phi):
        F = ngsolve.LinearForm(V)
        F += (1 / tau) * gfu_phi * v_phi * dx  
        F.Assemble()
        return F

    gfu = ngsolve.GridFunction(V)  # Mixed solution vector [phi, mu]

    Draw(gfu_phi)
    # Time-stepping loop
    for n in range(iter):
        # Update RHS
        F = assemble_F(gfu_phi)
        A = assemble_A(gfu_phi)
        
        # Solve the coupled system
        gfu.vec.data = A.mat.Inverse(V.FreeDofs()) * F.vec

        # Extract phi and mu components
        gfu_phi.vec.data = gfu.components[0].vec
        gfu_mu.vec.data = gfu.components[1].vec


    Draw(gfu_phi)

In [ ]:
h_max = 0.05
tau = 1e-5

IMEX(h_max,tau, iter = 100, fix_seed=True)

Fully Implicit

$$
\begin{align*}
\left( \frac{\phi^{n+1} - \phi^n}{\tau}, v \right) + (M \nabla \mu^{n+1}, \nabla v) &= 0 \quad \text{for all } v \in V, \\
(\mu^{n+1}, w) - \left( \gamma ((\phi^{n+1})^3 - \phi^{n+1}), w \right) - (\epsilon \nabla \phi^{n+1}, \nabla w) &= 0 \quad \text{for all } w \in V.
\end{align*}

$$


In [8]:
# Fully implicit


def fullyImplicit(h_max, tau,iter, tol, max_newton_iter = 100, fix_seed = False):

    # Mesh
    mesh = genMesh(h_max)

    # Defining finite element space

    V_phi = ngsolve.H1(mesh,order=1)
    V_mu  = ngsolve.H1(mesh,order=1)

    V = ngsolve.FESpace([V_phi, V_mu])  # Mixed finite element space
    (u_phi, u_mu), (v_phi, v_mu) = V.TnT()  # Trial and test functions


    gfu = ngsolve.GridFunction(V)
    gfu_phi = gfu.components[0]
    gfu_mu = gfu.components[1]

    gfu_phi_old = ngsolve.GridFunction(V_phi)


    # Initialisation of phi_0
    if fix_seed:
        np.random.seed(0)
    phi_0 = np.random.uniform(low = -1, high = 1, size = mesh.nv)
    rand_expr = ngsolve.GridFunction(V_phi)
    for i in range(mesh.nv):
        rand_expr.vec[i] = phi_0[i]

    gfu_phi.Set(rand_expr) 

    # Residual
    def assemble_Residual(gfu, gfu_phi_old):
        Residual = ngsolve.LinearForm(V)
        Residual += (1 / tau) * (gfu.components[0] - gfu_phi_old) * v_phi * dx  
        Residual += M * grad(gfu.components[1]) * grad(v_phi) * dx             
        Residual += gfu.components[1] * v_mu * dx                             
        Residual += -epsilon * grad(gfu.components[0]) * grad(v_mu) * dx      
        Residual += -gamma * gfu.components[0] * (gfu.components[0] - 1) * (gfu.components[0] + 1) * v_mu * dx  
        Residual.Assemble()
        return Residual



    # Jacobian components
    J_phi_phi = (1 / tau) * u_phi * v_phi * dx  # Time-stepping term
    J_phi_mu = M * grad(u_mu) * grad(v_phi) * dx  # Flux term
    J_mu_phi = -epsilon * grad(u_phi) * grad(v_mu) * dx  # Diffusive term
    J_mu_mu = u_mu * v_mu * dx  # Implicit mu term
    J_mu_nonlinear = -gamma * (3 * u_phi**2 - 1) * v_mu * dx  # Non-linear term

    # Total Jacobian
    Jacobian = ngsolve.BilinearForm(V)
    Jacobian += J_phi_phi + J_phi_mu + J_mu_phi + J_mu_mu + J_mu_nonlinear
    Jacobian.Assemble()

    

    Draw(gfu.components[0], mesh, "phi")
    for n in range(iter):
        gfu_phi_old.vec.data = gfu.components[0].vec  # Store old phi solution

        # Newton iterations
        for k in range(max_newton_iter):
            Residual = assemble_Residual(gfu,gfu_phi_old)

            # Compute Newton correction
            delta = Jacobian.mat.Inverse(V.FreeDofs()) * Residual.vec
            gfu.vec.data -= delta  # Update solution
            

            if delta.Norm() < tol:
                break
        else:
            raise RuntimeError(f"Newton did not converge! Norm: {delta.Norm()}")
        
    Draw(gfu.components[0], mesh, "phi")


In [ ]:
tau = 1e-5
h_max
iter = 100
max_newton_iter = 1000
tol = 1e-4

fullyImplicit(h_max,tau,iter,tol,max_newton_iter,fix_seed = True)



**Task 2.4**

To test correctness we consider a manufactured solution such that we get equations of the following form.

$$
\begin{align*}
        \left( \frac{\partial \phi}{\partial t}, v \right) + (M \nabla \mu, \nabla v) &= (f, v) \quad \text{for all } v \in V \\

        (\mu, w) - \left( \gamma (\phi^3 - \phi), w \right) - (\epsilon \nabla \phi, \nabla w) &= (g, w) \quad \text{for all } w \in V
\end{align*}



$$

 Since we have the boundary conditions 

$$
\nabla\phi \cdot \hat{n} = 0, \quad \nabla\mu \cdot \hat{n} = 0
$$

we choose the solutions
$$
\begin{equation*}
\phi(x, y, t) = \cos(\pi x) \cos(\pi y) e^{-t} = \mu(x, y, t)
\end{equation*}
$$

With this expression we have the following identities

$$
\partial_t\phi = -\phi, \quad \Delta\phi = -2\pi^2\phi, \quad \Delta\mu = -2\pi^2\mu
$$

We then insert it into the original split differential equation given by 

$$
\begin{align*}
    \partial_t \phi &= \nabla \cdot [M \nabla \mu] \quad \text{in } \Omega \times [0, T] \\
    \mu &= \gamma (\phi^3 - \phi) - \epsilon \Delta \phi \quad \text{in } \Omega \times [0, T]
\end{align*}

$$

from which we get

$$
\begin{align*}
    f &= \partial_t \phi - \nabla \cdot [M \nabla \mu] = -\phi + 2\pi^2 M \mu\\
    g &= \mu - \gamma (\phi^3 - \phi) + \epsilon \Delta \phi = \mu - \gamma (\phi^3 - \phi) - 2\pi^2\epsilon \phi


\end{align*}

$$

In [10]:
# Exact manufactured solution

def phi_exact(x,y,t): 
    return cos(pi*x)*cos(pi*y)*exp(-t)
def mu_exact(x,y,t): 
    return cos(pi*x)*cos(pi*y)*exp(-t)

def f_func(x,y,t):
    return -phi_exact(x,y,t) + 2*pi**2*M*mu_exact(x,y,t)
def g_func(x,y,t):
    return mu_exact(x,y,t) - gamma*(phi_exact(x,y,t)**3-phi_exact(x,y,t))- 2*pi**2*epsilon*phi_exact(x,y,t)


In [41]:
def fullyExplicitModified(h_max,tau, iter):
    mesh= genMesh(h_max)

    # Defining finite element space

    V_phi = ngsolve.H1(mesh,order=1, dirichlet= "boundary")
    V_mu  = ngsolve.H1(mesh,order=1, dirichlet= "boundary")

    V = ngsolve.FESpace([V_phi, V_mu])  # Mixed finite element space
    (u_phi, u_mu), (v_phi, v_mu) = V.TnT()  # Trial and test functions

    gfu_phi = ngsolve.GridFunction(V_phi)
    gfu_mu = ngsolve.GridFunction(V_mu)


    # Boundary conditions
    t = 0 # Must be updated over time, 

    def phi_red(x,y):
        return phi_exact(x,y,t)
    def mu_red(x,y):
        return mu_exact(x,y,t)

    phi_boundary = ngsolve.CoefficientFunction(phi_red(x,y)) 
    mu_boundary = ngsolve.CoefficientFunction(mu_red(x,y))

    gfu_phi.Set(phi_boundary, definedon=mesh.Boundaries("boundary"))
    gfu_mu.Set(mu_boundary, definedon=mesh.Boundaries("boundary"))

    # Set the boundary values on the GridFunctions
    gfu_phi.Set(phi_boundary, definedon=mesh.Boundaries("boundary"))
    gfu_mu.Set(mu_boundary, definedon=mesh.Boundaries("boundary"))


    # Exact grid function
    gfu_phi_exact = ngsolve.GridFunction(V_phi)
    gfu_phi_exact.Set(phi_boundary)



    # Bilinear form

    def assemble_A(gfu_phi):
        A = ngsolve.BilinearForm(V)
        A += (1 / tau) * u_phi * v_phi * dx  
        A += M * grad(u_mu) * grad(v_phi) * dx  
        A += u_mu * v_mu * dx  
        A += -epsilon * grad(u_phi) * grad(v_mu) * dx   
        A.Assemble()
        return A

    # linear form

    def assemble_F(gfu_phi):
        F = ngsolve.LinearForm(V)
        F += (1 / tau) * gfu_phi * v_phi * dx  
        F += gamma * (gfu_phi**3 - gfu_phi)* v_mu * dx 

        # Contribution of f and g
        F += f_func(x,y,t) * v_phi * dx
        F += g_func(x,y,t) * v_mu * dx

        F.Assemble()
        return F

    gfu = ngsolve.GridFunction(V)  # Mixed solution vector [phi, mu]

    Draw(gfu_phi_exact)
    # Time-stepping loop
    for n in range(iter):
        # Update time
        t = n*tau

        # Set the boundary values on the GridFunctions
        def phi_red(x,y):
            return phi_exact(x,y,t)
        def mu_red(x,y):
            return mu_exact(x,y,t)

        phi_boundary = ngsolve.CoefficientFunction(phi_red(x,y)) 
        mu_boundary = ngsolve.CoefficientFunction(mu_red(x,y))

        gfu_phi.Set(phi_boundary, definedon=mesh.Boundaries("boundary"))
        gfu_mu.Set(mu_boundary, definedon=mesh.Boundaries("boundary"))

        gfu_phi.Set(phi_boundary, definedon=mesh.Boundaries("boundary"))
        gfu_mu.Set(mu_boundary, definedon=mesh.Boundaries("boundary"))

        # Update RHS
        F = assemble_F(gfu_phi)
        A = assemble_A(gfu_phi)
        
        # Solve the coupled system
        gfu.vec.data = A.mat.Inverse(V.FreeDofs()) * F.vec

        # Extract phi and mu components
        gfu_phi.vec.data = gfu.components[0].vec
        gfu_mu.vec.data = gfu.components[1].vec


    Draw(gfu_phi)
    Draw(gfu_mu)


In [ ]:
h_max = 0.1
tau = 1e-5

fullyExplicitModified(h_max,tau,iter = 2)

# The plots are the exact solution of phi (and mu), the computed solution of phi and the computed solution of mu
